# Подключение библиотек

In [3]:
from bs4 import BeautifulSoup as bs

In [5]:
import requests

In [6]:
import pandas as pd

In [7]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [11]:
url = 'https://www.kinonews.ru/top100/'
page = requests.get(url)

In [12]:
page.status_code

200

In [13]:
soup = bs(page.text, 'html.parser')

In [17]:
page.text

'<!DOCTYPE html>\n<html lang="ru">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n<meta name="viewport" content="width=device-width, initial-scale=1, user-scalable=yes">\n<link rel="icon" href="https://www.kinonews.ru/favicon.ico" type="image/x-icon">\n<link rel="shortcut icon" href="https://www.kinonews.ru/favicon.ico" type="image/x-icon">\n<meta property="twitter:domain" content="KinoNews.ru">\n<meta property="twitter:card" content="summary_large_image">\n<meta property="twitter:site" content="@KinoNewsRu">\n<meta name="twitter:creator" content="@KinoNewsRu">\n<meta name="twitter:card" content="summary_large_image">\n<title>💯 Топ 100 лучших фильмов на KinoNews.ru. Полный список.</title>\n<meta name=\'title\' content=\'💯 Топ 100 лучших фильмов на KinoNews.ru. Полный список.\'>\n<meta name=\'keywords\' content=\'топ 100 лучших фильмов, топ 100 фильмов, top 100, фильмы с самым высоким рейтингом, фильмы по рейтингу, подборка фильмов, топ 100 лучших фильмов 

In [19]:
soup

<!DOCTYPE html>

<html lang="ru">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width, initial-scale=1, user-scalable=yes" name="viewport"/>
<link href="https://www.kinonews.ru/favicon.ico" rel="icon" type="image/x-icon"/>
<link href="https://www.kinonews.ru/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<meta content="KinoNews.ru" property="twitter:domain"/>
<meta content="summary_large_image" property="twitter:card"/>
<meta content="@KinoNewsRu" property="twitter:site"/>
<meta content="@KinoNewsRu" name="twitter:creator"/>
<meta content="summary_large_image" name="twitter:card"/>
<title>💯 Топ 100 лучших фильмов на KinoNews.ru. Полный список.</title>
<meta content="💯 Топ 100 лучших фильмов на KinoNews.ru. Полный список." name="title"/>
<meta content="топ 100 лучших фильмов, топ 100 фильмов, top 100, фильмы с самым высоким рейтингом, фильмы по рейтингу, подборка фильмов, топ 100 лучших фильмов всех времен, самые популярн

In [21]:
result_list = {
    'title': [],
    'year': [],
    'duration': [],
    'description': [],
    'genre': [],
    'country': []
}

In [23]:
pagenum = 0

for i in range(10):
    url = f'https://www.kinonews.ru/top100_p{pagenum}/'
    page = requests.get(url, headers=headers)

    if page.status_code != 200:
        print(f"Ошибка: статус код {page.status_code}")
        break

    soup = bs(page.text, 'html.parser')
    movies = soup.find_all('div', class_='bigtext')
    
    for movie in movies:
        title_element = movie.find('a', class_='titlefilm')
        title = title_element.text.strip() if title_element else 'Нет названия'
        movie_url = title_element.get('href') if title_element else None
        
        year = movie.text.split(', ')[-1].strip() if movie else 'Нет данных о годе'
        
        if movie_url:
            # Полный URL фильма
            movie_url = f'https://www.kinonews.ru{movie_url}'
            movie_page = requests.get(movie_url, headers=headers)
            if movie_page.status_code != 200:
                print(f"Ошибка при получении страницы фильма: статус код {movie_page.status_code}")
                continue

            movie_soup = bs(movie_page.text, 'html.parser')

            # Извлечение данных со страницы фильма
            description_element = movie_soup.find('div', itemprop='description')
            description = description_element.text.strip() if description_element else 'Нет описания на странице'

            genre_elements = movie_soup.find("td", class_="textgray")
            genres_middle = genre_elements.find_all("span", itemprop="genre")
            genres = ', '.join(genre.text.strip() for genre in genres_middle) if genres_middle else 'Нет данных о жанрах'

            country_elements = movie_soup.find_all("meta", itemprop="countryOfOrigin")
            countries = ', '.join(country['content'] for country in country_elements if 'content' in country.attrs)
            country = countries if countries else 'Нет данных о стране'

            # Получение продолжительности
            duration = 'Нет данных о продолжительности'
            table = movie_soup.find("table", class_="tab-film")

            if table:
                rows = table.find_all("tr")
                for row in rows:
                    cells = row.find_all("td")
                    if len(cells) > 1 and "Продолжительность:" in cells[0].text:
                        duration = cells[1].text.strip()
                        break

            # Добавление данных в результат
            result_list['title'].append(title)
            result_list['year'].append(year)
            result_list['description'].append(description)
            result_list['genre'].append(genres)
            result_list['country'].append(country)
            result_list['duration'].append(duration)

    pagenum += 1

# Вывод результатов
for i in range(len(result_list['title'])):
    print(f"Название: {result_list['title'][i]}, Год: {result_list['year'][i]}, "
          f"Жанр: {result_list['genre'][i]}, Страна: {result_list['country'][i]}, "
          f"Продолжительность: {result_list['duration'][i]}, Описание: {result_list['description'][i]}")

Название: Властелин колец 3: Возвращение Короля, Год: 2003, Жанр: боевик, приключения, фэнтези, Страна: США, Новая Зеландия, Продолжительность: 201 мин., Описание: Близится последняя битва за Средиземье. На пути войск Саурона стоит лишь королевство Гондор, которым управляет выживший из ума наместник Денетор. Силы хранителя кольца иссякают по мере продвижения к Мордору. Еще чуть-чуть и мир будет повержен во тьму.
Название: Побег из Шоушенка, Год: 1994, Жанр: драма, Страна: США, Продолжительность: 142 мин., Описание: "Побег из Шоушенка" - фильм, который считается одним из лучших в истории кино. Режиссером выступил Фрэнк Дарабонт, а в главных ролях снялись Тим Роббинс и Морган Фриман. Фильм основан на рассказе Стивена Кинга и повествует о бухгалтере Энди Дюфрейне, который осужден за убийство жены и ее любовника. Он отправляется в тюрьму Шоушенк, где сталкивается со многими трудностями и проблемами, но не теряет надежды на свободу. "Побег из Шоушенка" провалился в мировом прокате, был номи

In [24]:
print("Количество нулевых значений в: ")
for i in result_list:
    print( i + " - " + str(result_list[i].count(None)))

Количество нулевых значений в: 
title - 0
year - 0
duration - 0
description - 0
genre - 0
country - 0


In [25]:
file_name = 'kinonews.csv'
df = pd.DataFrame(data=result_list)
df.to_csv(file_name)

In [26]:
df.head()

,title,year,duration,description,genre,country
0,Властелин колец 3: Возвращение Короля,2003,201 мин.,Близится последняя битва за Средиземье. На пут...,"боевик, приключения, фэнтези","США, Новая Зеландия"
1,Побег из Шоушенка,1994,142 мин.,"""Побег из Шоушенка"" - фильм, который считается...",драма,США
2,Терминатор 2: Судный день,1991,137 мин.,Первая попытка убить Сару Коннор еще до рожден...,"боевик, триллер, фантастика",США
3,Форрест Гамп,1994,142 мин.,Форресту Гампу - умственно отсталому человеку ...,"драма, комедия, мелодрама",США
4,Властелин колец 2: Две крепости,2002,179 мин.,Братство кольца распалось. Фродо и Сэм продолж...,"боевик, приключения, фэнтези","США, Новая Зеландия"


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        450 non-null    object
 1   year         450 non-null    object
 2   duration     450 non-null    object
 3   description  450 non-null    object
 4   genre        450 non-null    object
 5   country      450 non-null    object
dtypes: object(6)
memory usage: 21.2+ KB
